In [13]:
# --------------------------------------------------------------------------
# Imports
# --------------------------------------------------------------------------

import os
import datetime

from windowgenerator import WindowGenerator
from models import Baseline

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sqlalchemy import create_engine, sql

In [14]:
# matplotlib settings
mpl.rcParams["figure.figsize"] = (5, 3)
mpl.rcParams["axes.grid"] = False

In [15]:
# --------------------------------------------------------------------------
# Database connection setup
# --------------------------------------------------------------------------

# get postgres environment variables
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DB_NAME = os.getenv("PG_DB_NAME")
PG_USERNAME = os.getenv("PG_USERNAME")
PG_PASSWORD = os.getenv("PG_PASSWORD")

# check for missing environment variables
if (
    PG_HOST == None
    or PG_PORT == None
    or PG_DB_NAME == None
    or PG_USERNAME == None
    or PG_PASSWORD == None
):
    print(
        "[ ERROR ] Environment variables PG_HOST, PG_PORT, PG_DB_NAME, PG_USERNAME, or PG_PASSWORD not found."
    )

# hard-code table name
PG_TABLE = "bars_minute_eastern"

# connect to db and open a cursor to perform database operations
conn_string = "postgresql://{}:{}@{}:{}/{}".format(
    PG_USERNAME, PG_PASSWORD, PG_HOST, PG_PORT, PG_DB_NAME
)
db = create_engine(conn_string)
conn = db.connect()

In [16]:
# -----------------------------------------------------------------------------
# Pull data from db and wrangle into correct shape
# (minute bars for NVDA from 09:00-14:00 from 2022/06/01-2022/07/01)
# -----------------------------------------------------------------------------

# create empty dataframe to hold bar data
days_data_df = pd.DataFrame(columns=["date", "minute_bars"])

# create empty dataframe to hold timestamp data
days_timestamps_df = pd.DataFrame(columns=["date", "timestamp"])

# NOTE: db contains minute bars for NVDA,INTC from 2022/06/01 to 2022/07/01 (inclusive)
start_date = datetime.date(2022, 6, 1)
end_date = datetime.date(2022, 7, 1)

# loop over each date and get minute_bars from db
current_date = start_date
day_delta = datetime.timedelta(days=1)
while current_date <= end_date:
    # pull minute_bars for current_date
    day_minute_bars_df = pd.read_sql_query(
        sql=sql.text(
            f"select * from { PG_TABLE } where SYMBOL='NVDA' and date(TIMESTAMP)='{ current_date }'"
        ),
        con=conn,
    )

    # add minute_bars to days_df if there is bar data
    if not day_minute_bars_df.empty:
        # drop symbol column
        day_minute_bars_df.drop("symbol", axis=1, inplace=True)

        # ---------------------------------------------------------------------
        # Convert timestamps to day/week/year sin and cosine signals
        # ---------------------------------------------------------------------

        # extract timestamps
        timestamps = pd.to_datetime(day_minute_bars_df.pop("timestamp"))

        timestamp_s = timestamps.map(pd.Timestamp.timestamp)

        seconds_per_day = 24 * 60 * 60
        seconds_per_week = seconds_per_day * 7
        seconds_per_year = seconds_per_week * 52.1429

        day_minute_bars_df["day-sin"] = np.sin(
            timestamp_s * (2 * np.pi / seconds_per_day)
        )
        day_minute_bars_df["day-cos"] = np.cos(
            timestamp_s * (2 * np.pi / seconds_per_day)
        )

        day_minute_bars_df["week-sin"] = np.sin(
            timestamp_s * (2 * np.pi / seconds_per_week)
        )
        day_minute_bars_df["week-cos"] = np.cos(
            timestamp_s * (2 * np.pi / seconds_per_week)
        )

        day_minute_bars_df["year-sin"] = np.sin(
            timestamp_s * (2 * np.pi / seconds_per_year)
        )
        day_minute_bars_df["year-cos"] = np.cos(
            timestamp_s * (2 * np.pi / seconds_per_year)
        )

        # insert day minute bars and timestamps to their respecive dataframes
        days_data_df.loc[len(days_data_df), days_data_df.columns] = (
            current_date,
            day_minute_bars_df,
        )
        days_timestamps_df.loc[len(days_timestamps_df), days_timestamps_df.columns] = (
            current_date,
            timestamps,
        )

    # go to next day
    current_date += day_delta

In [27]:
# -----------------------------------------------------------------------------
# Split into training (70%), validation (20%), and test (10%) sets
# -----------------------------------------------------------------------------

n = len(days_data_df)  # total number of days
days_train_df = days_data_df[0 : int(n * 0.7)]
days_val_df = days_data_df[int(n * 0.7) : int(n * 0.9)]
days_test_df = days_data_df[int(n * 0.9) :]

In [35]:
# -----------------------------------------------------------------------------
# Create baseline model (single-step, label is input)
# -----------------------------------------------------------------------------

current_date = start_date
while current_date <= end_date:
    # skip empty days (likely weekend or holiday)
    if days_data_df[days_data_df['date'] == current_date].empty:
        current_date += day_delta
        continue

    # pull minute_bars for current_date
    day_minute_bars_df = days_data_df[days_data_df['date'] == current_date]['minute_bars'].iloc[0]

    

    current_date += day_delta